In [1]:
# importando bibliotecas necessárias

import pandas as pd
import numpy as np

In [2]:
# carregando dados

dataset = pd.read_excel('/content/Data - Survey Monkey Output.xlsx')

In [3]:
# criando cópia dos dados para editá-los

df = dataset.copy()

In [4]:
# verificando linhas e colunas

df.shape

(199, 100)

In [5]:
# exibindo amostra dos dados

df.head()

,Respondent ID,Start Date,End Date,Email Address,First Name,Last Name,Custom Data 1,Identify which division you work in.,Unnamed: 8,Which of the following best describes your position level?,...,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Question 30,Unnamed: 98,Unnamed: 99
0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,Response,Other (please specify),Response,...,Response 8,Response 9,Response 10,Response 11,Response 12,Response 13,Response 14,Response 1,Response 2,Response 3
1,5.379192e+09,2021-01-22 12:01:17,2021-01-22 12:40:34,NaN,NaN,NaN,NaN,Infrastructure,NaN,Staff,...,NaN,Answer 8,Answer 8,Answer 4,NaN,NaN,Answer 5,NaN,NaN,NaN
2,2.658723e+09,2021-01-22 06:56:37,2021-01-22 07:34:10,NaN,NaN,NaN,NaN,Finance,NaN,Staff,...,NaN,Answer 5,NaN,NaN,Answer 2,NaN,Answer 5,NaN,NaN,Answer 1
3,4.044163e+09,2021-01-22 06:35:18,2021-01-22 06:47:32,NaN,NaN,NaN,NaN,Infrastructure,NaN,Department Lead,...,NaN,NaN,Answer 4,Answer 4,Answer 6,NaN,Answer 6,NaN,Answer 1,NaN
4,5.535866e+09,2021-01-21 21:29:32,2021-01-21 21:40:24,NaN,NaN,NaN,NaN,Infrastructure,NaN,Manager,...,Answer 2,Answer 5,Answer 7,NaN,Answer 6,NaN,Answer 7,Answer 7,Answer 1,Answer 6


In [6]:
# eliminando colunas desnecessárias

df.drop(['Start Date', 'End Date', 'Email Address', 'First Name', 'Last Name', 'Custom Data 1'], axis = 1 , inplace = True )

In [7]:
# renomeando colunas que correspondem a condição

def fixingColumns( cols : list ):
  # loop para testar condição em cada nome de coluna
  for index, col in enumerate(cols):
    # caso a coluna comece com 'Unnamed' renomeie a mesma para o nome da coluna anterior
    if col.startswith("Unnamed"):
      cols[index] = cols[index-1]
  return cols

# renomeando todas colunas para os novos nomes

df.columns = fixingColumns(list(df.columns))

In [8]:
# unindo valores da coluna e primeira linha (subtítulo das colunas na tabela importada) exceto por valores nulos

df.columns = df.columns + " " + df.iloc[0].fillna("")

In [9]:
# deletando primeira linha

df.drop( 0 , axis=0 , inplace = True )

In [10]:
# limpando espaços em branco ao redor dos valores das colunas

df.columns = [col.strip() for col in df.columns]

In [11]:
# definindo onde iniciar fatia de colunas para transpor

n = list(df.columns).index('Question 1 Response')

In [12]:
# selecionando colunas que terão valores transpostos

subset = list(df.columns[n:])

# transpondo dados

melted_df = pd.melt(df, id_vars = df.drop(subset, axis = 1), value_vars = subset, var_name = "Pergunta + Subpergunta", value_name = "Resposta")

# exibindo resultado

melted_df.head()

,Respondent ID,Identify which division you work in. Response,Identify which division you work in. Other (please specify),Which of the following best describes your position level? Response,Which generation are you apart of? Response,Please select the gender in which you identify. Response,Which duration range best aligns with your tenure at your company? Response,Which of the following best describes your employment type? Response,Pergunta + Subpergunta,Resposta
0,5.379192e+09,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1 Response,NaN
1,2.658723e+09,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Question 1 Response,Answer 4
2,4.044163e+09,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 Response,Answer 5
3,5.535866e+09,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1 Response,Answer 1
4,3.356803e+09,Port Operations,NaN,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1 Response,NaN


In [13]:
# adicionando coluna de questão para efetuar agrupamentos

melted_df['Pergunta'] = melted_df["Pergunta + Subpergunta"].apply(lambda x: x.split('R')[0])

In [14]:
# eliminando linhas com perguntas não respondidas

validAnswers = melted_df.dropna(axis = 0, subset = 'Resposta')

# criando contador de respostas por perguntas

validAnswers = validAnswers.groupby('Pergunta')['Respondent ID'].nunique().reset_index()

# exibindo quantidade de respostas obtidas por pergunta

validAnswers

,Pergunta,Respondent ID
0,Question 1,119
1,Question 10,198
2,Question 11,164
3,Question 12,114
4,Question 13,108
5,Question 14,105
6,Question 15,114
7,Question 16,117
8,Question 17,135
9,Question 18,109


In [15]:
# renomeando coluna para Entrevistados para evitar duplicação ao adicionar ao dataframe final

validAnswers.rename(columns = {'Respondent ID' : 'Entrevistados'}, inplace = True)

In [16]:
# criando contador de respostas iguais obtidas

qtyAnswers = melted_df.groupby(['Pergunta', 'Resposta'])['Respondent ID'].count().reset_index()

# exibindo quantidade de respostas iguais obtidas

qtyAnswers

,Pergunta,Resposta,Respondent ID
0,Question 1,Answer 1,14
1,Question 1,Answer 2,10
2,Question 1,Answer 3,13
3,Question 1,Answer 4,17
4,Question 1,Answer 5,22
...,...,...,...
259,Question 9,Answer 4,47
260,Question 9,Answer 5,60
261,Question 9,Answer 6,59
262,Question 9,Answer 7,62


In [17]:
# renomeando coluna para qtdRespostas para evitar duplicação ao adicionar ao dataframe final

qtyAnswers.rename(columns = {'Respondent ID' : 'qtdRespostasIguais'}, inplace = True)

In [20]:
# unindo contagem de perguntas respondidas e respostas iguais ao dataframe

final_df = pd.merge(left = melted_df, right = validAnswers, how = 'left', left_on = 'Pergunta', right_on = 'Pergunta')

final_df = pd.merge(left = final_df, right = qtyAnswers, how = 'left', left_on = ['Pergunta', 'Resposta'], right_on = ['Pergunta', 'Resposta'])

# preenchendo valores nulos da coluna qtdRespostasIguais com 0

final_df['qtdRespostasIguais'].fillna(value = 0, inplace = True)

# exibindo resultado final da transformação

final_df

,Respondent ID,Identify which division you work in. Response,Identify which division you work in. Other (please specify),Which of the following best describes your position level? Response,Which generation are you apart of? Response,Please select the gender in which you identify. Response,Which duration range best aligns with your tenure at your company? Response,Which of the following best describes your employment type? Response,Pergunta + Subpergunta,Resposta,Pergunta,Entrevistados,qtdRespostasIguais
0,5.379192e+09,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1 Response,NaN,Question 1,119,NaN
1,2.658723e+09,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Question 1 Response,Answer 4,Question 1,119,17.0
2,4.044163e+09,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 Response,Answer 5,Question 1,119,22.0
3,5.535866e+09,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1 Response,Answer 1,Question 1,119,14.0
4,3.356803e+09,Port Operations,NaN,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1 Response,NaN,Question 1,119,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17023,7.940065e+09,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30 Response 3,Answer 8,Question 30,182,42.0
17024,5.157706e+09,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30 Response 3,Answer 6,Question 30,182,53.0
17025,9.920756e+09,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 Response 3,NaN,Question 30,182,NaN
17026,6.638341e+09,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 Response 3,NaN,Question 30,182,NaN


In [24]:
# criando cópia do dataframe para exportar

output = final_df.copy()

In [25]:
# renomeando colunas para melhor visualização

output.columns

Index(['Respondent ID', 'Identify which division you work in. Response',
       'Identify which division you work in. Other (please specify)',
       'Which of the following best describes your position level? Response',
       'Which generation are you apart of? Response',
       'Please select the gender in which you identify. Response',
       'Which duration range best aligns with your tenure at your company? Response',
       'Which of the following best describes your employment type? Response',
       'Pergunta + Subpergunta', 'Resposta', 'Pergunta', 'Entrevistados',
       'qtdRespostasIguais'],
      dtype='object')

In [26]:
output.rename({'Identify which division you work in. Response' : 'Divisão Primária',
       'Identify which division you work in. Other (please specify)' : 'Divisão Secundária',
       'Which of the following best describes your position level? Response' : 'Posição',
       'Which generation are you apart of? Response' : 'Geração',
       'Please select the gender in which you identify. Response' : 'Gênero',
       'Which duration range best aligns with your tenure at your company? Response' : 'Mandato',
       'Which of the following best describes your employment type? Response' : 'Tipo de Cargo'},
       inplace = True)

In [27]:
# exportando dataframe para arquivo xlsx

output.to_excel('./Resultado_Final_Survey_Monkey.xlsx', index = False)